### Overview
This notebook walks your through the process of creating clips with LLM prompts using multimodal information of video. 

Pick a video, decide your prompt, generate a new clip ⚡️

It's as simple as it sounds.

If you want to go extra mile you can add Image Overlays or Audio overlays on these clips.

In [ ]:
# But first, let's install the dependecies.
!pip install -r requirements.txt

### Loading The Video

Before proceeding, ensure access to [VideoDB](https://videodb.io) API key. If not, sign up for API access on the respective platforms.

> Get your API key from [VideoDB Console](https://console.videodb.io). ( Free for first 50 uploads, **No credit card required** ) 🎉

You can either source a new video from YouTube or select an existing one from your VideoDB collection.


In [ ]:
import os
import videodb

from dotenv import load_dotenv


# TODO: setup .env file
load_dotenv()

# Connect to VideoDB
conn = videodb.connect()
coll = conn.get_collection()

# TODO: Add video_id if video already exists in the collection
video_id = os.getenv("MULTIMODAL_DEMO_VIDEO_ID")
video_url = "https://www.youtube.com/watch?v=NZGLHdcw2RM"

if not video_id:
    video = coll.upload(url=video_url)
else:
    video = coll.get_video(video_id)

print(f"video_id: {video.id}, name: {video.name}")
video.play()

### Indexing The Visual And Spoken Information

In this example, we are using a cricket explainer video. We will index it based on spoken words and scenes to support complex queries that require multimodal information.

Below cell will retrieve the transcript if it's already indexed; otherwise, it will first index the content and then set the transcript.

In [7]:
try:
    transcript = video.get_transcript()
    transcript_text = video.get_transcript_text()
except Exception:
    video.index_spoken_words()
    transcript = video.get_transcript()
    transcript_text = video.get_transcript_text()

Here, you can either provide the `scene_index_id` of the video, if available, or leave it blank to index the video for visual retrieval.

To know more about scene indexing click [here](https://docs.videodb.io/scene-index-guide-80).

In [ ]:
# Add scene_index_id here if already indexed.
scene_index_id = os.getenv("MULTIMODAL_DEMO_SCENE_INDEX_ID")

if not scene_index_id:
    scene_index_id = video.index_scenes(
        prompt="Summarize the essence of the scene in one or two concise sentences."
    )
scenes = video.get_scene_index(scene_index_id)
print(f"Video is indexed with scene_index_id {scene_index_id} and has {len(scenes)} scenes.")

### Inspecting The Indexed Scenes

In [ ]:
for scene in scenes:
    print(f"{scene['start']}-{scene['end']}: {scene['description']}")

### Get Results From Your Prompt

The `multimodal_promper` function in `video_prompter.py` takes the transcript and indexed scenes, combines them, chunks them, and then parallelly calls the LLM with user and system prompts to retrieve the desired matching scenes.

To create a clip using the `multimodal_promper` function from a video, it's crucial to craft a specific prompt that will help identify the most relevant segments for your use case. This prompt should highlight the themes, activity, or combination of verbal or visual cues you're interested in. 

The `multimodal_promper` will return sentences which are visual description from matched chunks. Review these to ensure they align with your needs. You can then use these segments to create your clip, focusing on the content that's most relevant to your use case.

In [ ]:
from video_prompter import multimodal_prompter

user_prompt = "find the scene explaining the cricket rules using infographics, remember rules with infographics."
# Here, we are only interested in the section of the video where cricket rules are explained using infographics.
# If we create a clip using only the spoken index data, we won't know where the infographics are.
# If we create a clip using only the visual index data, we may include additional infographics that aren't actually about the rules but might appear to be rule explanations based on visual information.
# By creating a clip using combined data, we achieve a much more precise intersection where the infographics are present, and the rules are being explained.
result = multimodal_prompter(transcript, scenes, user_prompt)

### Generate The Clip
To generate a clip, first we'll call `get_result_timestamps` from `video_prompter.py` it uses VideoDB's `keyword search` feature. We already leveraged the power of the LLM (Large Language Model) to identify relevant sentences. We'll use the search results to create a programmable video stream. Here's how you can approach this process:

We have the descriptions in the `results` variable. We input these keywords into VideoDB's keyword search feature. This function will search through the indexed scenes of your videos to find matches.

The search will return a SearchResult object, which contains detailed information about the found segments, including their timestamps, the text of the scene description.

**Create a Programmable Video Stream with Timeline:** With the specific segments identified, you can now use `build_video_timeline` from `video_prompter.py` to get the Timeline to create a new programmable video stream. The Timeline tool allows you to stitch together video segments based on the timestamps provided in the SearchResult object. You can arrange, cut, or combine these segments to craft a fresh video stream that focuses on your topic of interest.

In [ ]:
from videodb import play_stream
from videodb.timeline import Timeline

from video_prompter import get_result_timestamps, build_video_timeline

timeline = Timeline(conn)
result_timestamps = get_result_timestamps(video, result, scene_index_id=scene_index_id)
timeline, duration = build_video_timeline(video, result_timestamps, timeline)
stream = timeline.generate_stream()
play_stream(stream)

### Bonus - With Single Modality For Comparison

#### Clip Using Only Spoken Information
Since the transcript does not include information about infographics, not all infographics with rules are captured in the result.
Segments explaining how many days each format is played are missing.

In [8]:
from video_prompter import text_prompter

user_prompt = "find the scene explaining the cricket rules using infographics, remember rules with infographics."
text_result = text_prompter(transcript_text, user_prompt)

In [ ]:
timeline = Timeline(conn)
result_timestamps = get_result_timestamps(video, text_result, index_type="spoken_word")
timeline, duration  = build_video_timeline(video, result_timestamps, timeline)
stream = timeline.generate_stream()
play_stream(stream)

Since the model doesn't have information about the visual infographics, the segments toward the end, where the rules about overs are explained, are missing from the results.

#### Clip On Scene Information

Here, segments that contain infographics but are not related to rules are also being captured in the results too.

In [10]:
from  video_prompter import scene_prompter

user_prompt = "find the scene explaining the cricket rules using infographics, remember rules with infographics."
scene_result = scene_prompter(scenes, user_prompt)

In [ ]:
timeline = Timeline(conn)
result_timestamps = get_result_timestamps(video, scene_result, scene_index_id=scene_index_id)
timeline, duration = build_video_timeline(video, result_timestamps, timeline)
stream = timeline.generate_stream()
play_stream(stream)

Since the model doesn't have information about the spoken content, false positive segments are being added—these segments aren't actually about rules but may appear to be based on visual information.

### Conclusion 

As shown above, in cases where both modalities are important to retrieve the desired clip, this notebook works best. If only one modality is sufficient, the respective functions can be used.

We can't wait to see which approach works best for your videos—do share your results with us!

* [Discord](https://colab.research.google.com/corgiredirector?site=https%3A%2F%2Fdiscord.gg%2Fpy9P639jGz)
* [GitHub](https://github.com/video-db)
* [Website](https://colab.research.google.com/corgiredirector?site=https%3A%2F%2Fvideodb.io)
* [Email](ashu@videodb.io)

### Playground For You To Experiment

In [ ]:
video_url = ""
video_id = ""

if not video_id:
    video = coll.upload(url=video_url)
else:
    video = coll.get_video(video_id)

print(f"video_id: {video.id}, name: {video.name}")

In [ ]:
try:
    transcript = video.get_transcript()
    transcript_text = video.get_transcript_text()
except Exception:
    video.index_spoken_words()
    transcript = video.get_transcript()
    transcript_text = video.get_transcript_text()

In [ ]:
scene_index_id = ""

if not scene_index_id:
    scene_index_id = video.index_scenes(
        prompt="Summarize the essence of the scene in one or two concise sentences."
    )
scenes = video.get_scene_index(scene_index_id)
print(f"Video is indexed with scene_index_id {scene_index_id} and has {len(scenes)} scenes.")

In [ ]:
from video_prompter import multimodal_prompter

user_prompt = ""
result = multimodal_prompter(transcript, scenes, user_prompt)

In [ ]:
from videodb import play_stream
from videodb.timeline import Timeline

from video_prompter import get_result_timestamps, build_video_timeline

timeline = Timeline(conn)
result_timestamps = get_result_timestamps(video, result, scene_index_id=scene_index_id)
timeline, duration = build_video_timeline(video, result_timestamps, timeline)
stream = timeline.generate_stream()
play_stream(stream)